In [ ]:
import pandas as pd
import pysam
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from Bio.Seq import Seq

In [ ]:
Clonotype	Frequency	IGH_cdr3	IGH_cdr3_nt	contig_id_IGH	IGH_aa	IGH_nt	IGLorK_cdr3	IGLorK_cdr3_nt	contig_id_IGLorKs	IGLorK_aa	IGLorK_nt

In [ ]:
out_path = '/SGRNJ06/bioinfo/PROJ04/PROJ_22.10X/P22062202_VDJ/20220712/WCM-20220617-B_BCR/WCM-20220617-B_VDJ_10X/WCM-20220617-B_VDJ_10X/outs'

In [ ]:
airr = pd.read_csv(f'{out_path}/airr_rearrangement.tsv',sep='\t')
contig = pd.read_csv(f'{out_path}/filtered_contig_annotations.csv')
clonotype = pd.read_csv(f'{out_path}/clonotypes.csv')

In [ ]:
del clonotype['proportion']
clonotype

In [ ]:
clonotype['cdr3s_aa'] = clonotype['cdr3s_aa'].apply(lambda x:x.split(';'))
clonotype['cdr3s_nt'] = clonotype['cdr3s_nt'].apply(lambda x:x.split(';'))

In [ ]:
clonotype

In [ ]:
aa_list = clonotype['cdr3s_aa'].tolist()
aa_dict = {}
for i,j in enumerate(aa_list,1):
    aa_dict[i] = j

In [ ]:
aa_dict[3444]

In [ ]:
IGH_cdr3 = dict()
IGL_cdr3 = dict()

In [ ]:
for key,value in aa_dict.items():
    if len(value)==2: 
        for i in value:
            if i.startswith('IGH'):
                IGH_cdr3['clonotype'+str(key)]= (i.split(':')[-1])
            else:
                IGL_cdr3['clonotype'+str(key)]= (i.split(':')[-1])

In [ ]:
IGH_cdr3

In [ ]:
df_IGH_cdr3 = pd.DataFrame(list(IGH_cdr3.items()),columns=['clonotype_id', 'IGH_cdr3'])
df_IGL_cdr3 = pd.DataFrame(list(IGL_cdr3.items()),columns=['clonotype_id', 'IGLorK_cdr3'])

In [ ]:
df_merge = pd.merge(clonotype,df_IGH_cdr3,on='clonotype_id',how='inner')
df_merge = pd.merge(df_merge, df_IGL_cdr3,on='clonotype_id',how='inner')

In [ ]:
del df_merge['cdr3s_aa']

In [ ]:
df_merge

In [ ]:
nt_list = clonotype['cdr3s_nt'].tolist()
nt_dict = {}
for i,j in enumerate(nt_list,1):
    nt_dict[i] = j

In [ ]:
IGH_cdr3_nt = dict()
IGL_cdr3_nt = dict()

In [ ]:
for key,value in nt_dict.items():
    for i in value:
        if i.startswith('IGH'):
            IGH_cdr3_nt['clonotype'+str(key)]=(i.split(':')[-1])
        else:
            IGL_cdr3_nt['clonotype'+str(key)]=(i.split(':')[-1])

In [ ]:
df_IGH_cdr3_nt = pd.DataFrame(list(IGH_cdr3_nt.items()),columns=['clonotype_id', 'IGH_cdr3_nt'])
df_IGL_cdr3_nt = pd.DataFrame(list(IGL_cdr3_nt.items()),columns=['clonotype_id', 'IGLorK_cdr3_nt'])

In [ ]:
df_merge = pd.merge(df_merge,df_IGH_cdr3_nt,on='clonotype_id',how='inner')
df_merge = pd.merge(df_merge,df_IGL_cdr3_nt,on='clonotype_id',how='inner')

In [ ]:
del df_merge['cdr3s_nt']
df_merge

In [ ]:
df_merge['Clonotype'] = df_merge['IGH_cdr3'] + '_' + df_merge['IGLorK_cdr3']

In [ ]:
df_merge =df_merge[['clonotype_id','Clonotype','frequency','IGH_cdr3','IGH_cdr3_nt','IGLorK_cdr3','IGLorK_cdr3_nt']]

In [ ]:
df_merge

In [ ]:
airr

In [ ]:
airr = airr[['clone_id','sequence_id','sequence','sequence_aa']]

In [ ]:
airr.rename(columns={'sequence_id':'contig_id'},inplace=True)

In [ ]:
contig.sort_values('reads',ascending=False,inplace=True)

In [ ]:
contig= contig[['contig_id','chain']]

In [ ]:
df_merge1 = pd.merge(airr,contig,on='contig_id')

In [ ]:
df_merge1.rename(columns={'clone_id':'clonotype_id'},inplace=True)

In [ ]:
df_merge1['idnum'] = df_merge1['clonotype_id'].apply(lambda x: int(x[9:]))

In [ ]:
df_merge1.sort_values(['idnum','chain'],inplace=True)

In [ ]:
del df_merge1['idnum']

In [ ]:
df_merge1

In [ ]:
target_contigid = df_merge['clonotype_id'].tolist()

In [ ]:
df_merge1 = df_merge1[df_merge1['clonotype_id'].isin(target_contigid)]

In [ ]:
df_merge1

In [ ]:
df_merge1.drop_duplicates(['clonotype_id','chain'],inplace=True)
# df_merge1 = df_merge1.where(df_merge_f.notnull(),'None')

In [ ]:
df_merge1

In [ ]:
df_merge

In [ ]:
df_merge1_IGH = df_merge1[df_merge1['chain']=='IGH']
df_merge1_IGL = df_merge1[df_merge1['chain']!='IGH']

In [ ]:
del df_merge1_IGH['chain']
del df_merge1_IGL['chain']
df_merge1_IGH.rename(columns={'contig_id':'contig_id_IGH'},inplace=True)
df_merge1_IGL.rename(columns={'contig_id':'contig_id_IGLorKs'},inplace=True)

In [ ]:
df_merge1_IGH.rename(columns={'sequence':'IGH_nt'},inplace=True)
df_merge1_IGL.rename(columns={'sequence':'IGLorK_nt'},inplace=True)
df_merge1_IGH.rename(columns={'sequence_aa':'IGH_aa'},inplace=True)
df_merge1_IGL.rename(columns={'sequence_aa':'IGLorK_aa'},inplace=True)

In [ ]:
df_merge1_IGH

In [ ]:
df_merge_f = pd.merge(df_merge,df_merge1_IGH,on='clonotype_id')

In [ ]:
df_merge_f = pd.merge(df_merge_f,df_merge1_IGL,on='clonotype_id')

In [ ]:
df_merge_f

In [ ]:
df_merge_f.columns

In [ ]:
df_merge_f.rename(columns={'frequency':'Frequency'},inplace=True)

In [ ]:
df_merge_f = df_merge_f[['clonotype_id','Clonotype','Frequency','IGH_cdr3','IGH_cdr3_nt','contig_id_IGH','IGH_aa','IGH_nt',
                        'IGLorK_cdr3','IGLorK_cdr3_nt','contig_id_IGLorKs','IGLorK_aa','IGLorK_nt']]

In [ ]:
df_merge_f

In [ ]:
df_merge_f.to_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220725anno_seq_suzhou/filtered_contig_annotations_pair_productive_with_sequence.tsv',sep='\t')